In [ ]:
import pandas as pd
docentes = pd.read_csv("docentes.csv")

### 🧹 1. Unificación de docentes de Inglés

Este bloque estandariza los docentes que dictan Inglés a partir de 7mo EGB. Se eliminan los registros de las docentes "ELIZABETH PUGA" y "SIMONE AYALA", y se reemplaza a "DAISY HERRERA" por el identificador genérico `"INGLES ESPECIAL"`.

Este paso facilita la asignación conjunta de horarios en niveles superiores donde el docente de inglés es compartido entre paralelos.


In [ ]:
#Unificar docentes de inglés a partir de 7mo EGB
docentes=docentes.drop(docentes[docentes['Nombres']=='ELIZABETH PUGA'].index)
docentes=docentes.drop(docentes[docentes['Nombres']=='SIMONE AYALA'].index)
docentes["Nombres"] = docentes["Nombres"].replace("DAISY HERRERA", "INGLES ESPECIAL")

### 🏃 2. Unificación de docentes de Educación Física

De forma similar, se agrupan los docentes de Educación Física a partir de 3ro EGB. Se eliminan "CARLOS MORALES" e "IVÁN MALDONADO" y se reemplaza a "NANCY LINCANGO" por `"EF ESPECIAL"`.

Esto permite manejar la materia como una asignación especial compartida, lo cual es fundamental para las restricciones del modelo de horarios.


In [ ]:
#Unificar docentes de Educación Física a partir de 3ro EGB
docentes=docentes.drop(docentes[docentes['Nombres']=='CARLOS MORALES'].index)
docentes=docentes.drop(docentes[docentes['Nombres']=='IVÁN MALDONADO'].index)
docentes["Nombres"] = docentes["Nombres"].replace("NANCY LINCANGO", "EF ESPECIAL")

docentes.reset_index()

### 🧹 3. Normalización y expansión de códigos de asignación docente

En este bloque se transforma la columna `Codigo` del DataFrame `docentes`, que originalmente contiene múltiples códigos por celda, en un formato estructurado con una fila por cada asignación.

#### 🔄 Proceso aplicado:
1. **Copia del DataFrame:** Se trabaja sobre una copia (`df`) para mantener intacto el original.
2. **Limpieza de caracteres:** Se eliminan `{}`, `'` y otros símbolos innecesarios con una expresión regular.
3. **Separación y expansión:**
   - Se usa `.str.split(", ")` para dividir los códigos.
   - Se aplica `.explode()` para expandir los valores en filas separadas.

#### 📊 Resultado:
Cada fila del nuevo DataFrame `docentes` contiene una única asignación `{docente ↔ código}`.

Esto facilita la construcción de diccionarios rápidos y consultas eficientes para el algoritmo de generación de horarios.


In [ ]:
df=docentes.copy()
# Limpiar la columna 'Codigo': eliminar llaves y comillas simples
df["Codigo"] = df["Codigo"].str.replace(r"[{}']", "", regex=True)

# Separar los códigos por coma y expandir en filas individuales
docentes = df.assign(Codigo=df["Codigo"].str.split(", ")).explode("Codigo")

docentes.head(20)

,index,Nombres,Cargo,Materia,Codigo
0,0,MALENA OÑA,Tiempo completo,LENGUAJE,LEN_1EGB
1,1,ALISON PLAZA,Tiempo completo,LECTORES,LEC_1EGB
1,1,ALISON PLAZA,Tiempo completo,LECTORES,LEC_2EGB
1,1,ALISON PLAZA,Tiempo completo,LECTORES,LEC_3EGB
2,2,SOFÍA YÉPEZ,Tiempo completo,LENGUAJE,LEN_3EGB
2,2,SOFÍA YÉPEZ,Tiempo completo,LENGUAJE,LEN_2EGB
3,3,ROSITA ALBUJA,Tiempo completo,LENGUAJE,LEN_5EGB
3,3,ROSITA ALBUJA,Tiempo completo,LENGUAJE,LEN_4EGB
4,4,NARCISA MUÑOZ,Tiempo completo,LENGUAJE,LEN_6EGB
4,4,NARCISA MUÑOZ,Tiempo completo,LENGUAJE,LEN_7EGB


### 🧠 4. Construcción del diccionario de asignación docente (`dic_profesores`)

Este bloque itera sobre cada fila del DataFrame `docentes` y construye un diccionario llamado `dic_profesores` con el siguiente mapeo:

```python
{ código_materia → nombre_docente }

In [ ]:
df=docentes.copy()
# Carga del archivo CSV
df = docentes

# Identifica las columnas
col_codigo = 'Codigo'
col_profesor = 'Nombres'

# Diccionario resultado
dic_profesores = {}

for _, row in df.iterrows():
    raw = row[col_codigo]
    if pd.isna(raw):
        continue

    # Convertimos a cadena y eliminamos espacios exteriores
    s = str(raw).strip()

    # Si empieza por '{' o '[' acabamos quitando delimitadores
    if s.startswith('{') and s.endswith('}'):
        # Quitamos las llaves
        s = s[1:-1]
    elif s.startswith('[') and s.endswith(']'):
        # Quitamos los corchetes
        s = s[1:-1]

    # Ahora s es algo como "'LEC_1EGB', 'LEC_3EGB', 'LEC_2EGB'"
    # Partimos por comas y limpiamos comillas
    codes = []
    for part in s.split(','):
        part = part.strip().strip("'\"")  # quita comillas simples o dobles
        if part:
            codes.append(part)

    # Asociamos cada código con el profesor
    for code in codes:
        dic_profesores[code] = row[col_profesor]

# Ejemplo de uso:
for ejemplo in ['LEN_3EGB', 'PEN_10EGB', 'EST_10EGB']:
    prof = dic_profesores.get(ejemplo, 'No encontrado')
    print(f"Profesor de {ejemplo}: {prof}")

Profesor de LEN_3EGB: SOFÍA YÉPEZ
Profesor de PEN_10EGB: MIREYA BRITO
Profesor de EST_10EGB: JONATHAN CASTRO


### 🧑‍🏫 5. Función para obtener el docente a partir del código de materia

La función `obtener_docente_por_materia_codigo(...)` permite consultar qué docente imparte una determinada materia, usando el diccionario previamente construido `dic_profesores`.

#### 📌 Lógica:
- Recibe como parámetro el código completo de la materia, por ejemplo: `"MAT_3BGU"`, `"LEN_1EGB"`, `"EF_6EGB"`.
- Busca en `dic_profesores` y retorna el nombre del docente correspondiente.
- Si el código no existe en el diccionario, retorna `None`.

#### ✅ Aplicaciones:
Esta función es útil para:
- Asignar automáticamente docentes durante la generación de cromosomas.
- Validar asignaciones en un horario generado.
- Construir bloques LaTeX completos a partir de información parcial.


In [ ]:
# Obtener docente que imparte la materia
def obtener_docente_por_materia_codigo(codigo_materia):
    return dic_profesores.get(codigo_materia, None)

### 🧬 6. Reconstrucción de cromosoma global a partir de archivos `.txt`

Este bloque permite combinar múltiples horarios individuales (guardados como archivos `.txt`) en un solo cromosoma completo que sigue la estructura esperada por el algoritmo genético.

#### 📌 Componentes clave:

1. **Orden de niveles y días:**  
   Se define el orden deseado para garantizar una estructura coherente al ordenar el cromosoma.

2. **`leer_txt_como_tuplas(...)`:**  
   Función que lee un archivo `.txt` y retorna su contenido como lista de tuplas `(nivel, aula, día, hora, código_materia)`.

3. **`ordenar_cromosoma(...)`:**  
   Ordena el cromosoma combinando criterios jerárquicos: nivel → aula → día → hora.

4. **Listado de archivos:**  
   Se especifican los archivos `.txt` que contienen los horarios individuales por paralelo.

5. **`reconstruir_cromosoma_desde_txt(...)`:**  
   Carga todos los archivos, asigna automáticamente el docente a partir del código de materia usando `obtener_docente_por_materia_codigo`, y genera el cromosoma completo listo para ser evaluado o graficado.

#### ✅ Resultado:
Una lista estructurada de genes de la forma:



In [ ]:
import os

# === 1. ORDEN DE NIVELES Y DÍAS PARA POSTERIOR ORDENAMIENTO ===
orden_niveles = ['1EGB', '2EGB', '3EGB', '4EGB', '5EGB', '6EGB', '7EGB', '8EGB', '9EGB', '10EGB', '1BGU', '2BGU', '3BGU']
orden_dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes']

# === 2. FUNCIÓN PARA LEER TXT COMO LISTA DE TUPLAS ===
def leer_txt_como_tuplas(filepath):
    with open(filepath, "r", encoding="utf-8") as f:
        lines = f.readlines()
    return [eval(line.strip()) for line in lines if line.strip()]

# === 3. FUNCIÓN PARA ORDENAR CROMOSOMA (IGUAL A LA TUYA) ===
def ordenar_cromosoma(cromosoma):
    return sorted(
        cromosoma,
        key=lambda g: (
            orden_niveles.index(g[0]),
            g[1],
            orden_dias.index(g[2]),
            g[3]
        )
    )
# === 5. NOMBRES DE ARCHIVOS A INCLUIR ===
archivos_txt = [
    '1EGB_Ejemplares.txt', '1EGB_Entusiastas.txt', '1EGB_Estudiosos.txt', '1EGB_Exitosos.txt',
    '2BGU_Honestos.txt', '2BGU_Justos.txt', '2BGU_Proactivos.txt', '2EGB_Afectuosos.txt',
    '2EGB_Amables.txt', '2EGB_Amigables.txt', '2EGB_Amistosos.txt', '3BGU_Empáticos.txt',
    '3BGU_Solidarios.txt', '3BGU_Vanguardistas.txt', '3EGB_Colaboradores.txt', '3EGB_Conciliadores.txt',
    '3EGB_Cordiales.txt', '3EGB_Creativos.txt', '4EGB_Optimistas.txt', '4EGB_Ordenados.txt',
    '4EGB_Organizados.txt', '4EGB_Originales.txt', '5EGB_Admirables.txt', '5EGB_Afectivos.txt',
    '5EGB_Aplicados.txt', '5EGB_Atentos.txt', '6EGB_Ecuanimes.txt', '6EGB_Emprendedores.txt',
    '6EGB_Excelentes.txt', '6EGB_Expresivos.txt', '7EGB_Reflexivos.txt', '7EGB_Resilientes.txt',
    '7EGB_Respetuosos.txt', '7EGB_Responsables.txt', '8EGB_Decididos.txt', '8EGB_Diligentes.txt',
    '8EGB_Dinámicos.txt', '8EGB_Disciplinados.txt', '9EGB_Altruistas.txt', '9EGB_Asertivos.txt',
    '9EGB_Audaces.txt', '9EGB_Auténticos.txt', '10EGB_Pacifistas.txt', '10EGB_Perspicaces.txt',
    '10EGB_Positivos.txt', '1BGU_Confiables.txt', '1BGU_Íntegros.txt', '1BGU_Leales.txt'
]

# === 6. FUNCIÓN PARA GENERAR CROMOSOMA DESDE LOS TXT ===
def reconstruir_cromosoma_desde_txt():
    cromosoma = []

    for nombre_archivo in archivos_txt:
        if not os.path.exists(nombre_archivo):
            print(f"[!] Archivo no encontrado: {nombre_archivo}")
            continue

        datos = leer_txt_como_tuplas(nombre_archivo)
        for fila in datos:
            nivel, aula, dia, hora, codigo_materia = fila
            docente = obtener_docente_por_materia_codigo(codigo_materia)
            cromosoma.append((nivel, aula, dia, hora, codigo_materia, docente))

    return ordenar_cromosoma(cromosoma)

cromosoma = reconstruir_cromosoma_desde_txt()


### 🔍 7. Visualización y verificación del cromosoma reconstruido

Este bloque imprime información básica del cromosoma generado en el paso anterior.

#### 📌 Acciones realizadas:
- Se muestra el número total de genes generados, lo que equivale al total de bloques horarios asignados.
- Se imprimen todos los genes del cromosoma para inspeccionar manualmente su contenido.

Cada gen tiene la estructura:
```python
(nivel, aula, día, hora, código_materia, docente)


In [ ]:
print(f"Total de genes generados: {len(cromosoma)}")
# Mostrar los primeros 5 genes
for gen in cromosoma[:]:
    print(gen)

Total de genes generados: 2832
('1EGB', 'Ejemplares', 'Lunes', 1, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 2, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 3, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Lunes', 4, 'PEN_1EGB', 'FERNANDA CAÑIZARES')
('1EGB', 'Ejemplares', 'Lunes', 6, 'DES_1EGB', 'ANDREA MACIAS')
('1EGB', 'Ejemplares', 'Lunes', 7, 'EDU_1EGB', 'EF ESPECIAL')
('1EGB', 'Ejemplares', 'Lunes', 8, 'EDU_1EGB', 'EF ESPECIAL')
('1EGB', 'Ejemplares', 'Lunes', 9, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 10, 'MAT_1EGB', 'PAULINA ÑACATO')
('1EGB', 'Ejemplares', 'Lunes', 13, 'DAN_1EGB', 'PAOLA LÓPEZ')
('1EGB', 'Ejemplares', 'Lunes', 14, 'MUS_1EGB', 'LUIS JÁCOME')
('1EGB', 'Ejemplares', 'Martes', 1, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Martes', 2, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Martes', 3, 'LEN_1EGB', 'MALENA OÑA')
('1EGB', 'Ejemplares', 'Martes', 4, 'EST_1EGB', 'SANTIAGO CALLE')
('1E

### 🧪 8. Verificación de docentes asignados en el cromosoma

La función `verificar_docentes_asignados(cromosoma)` permite identificar genes que no tienen un docente asignado (es decir, aquellos donde `docente is None`).

#### 📌 Funcionalidad:
- Recorre cada gen del cromosoma.
- Detecta aquellos cuya posición de docente (`gene[5]`) está vacía (`None`).
- Imprime un resumen indicando cuántas asignaciones no tienen docente y detalla cada una.

#### ✅ Uso:
Permite validar que el diccionario `dic_profesores` esté completo y correctamente sincronizado con los códigos de materia usados en los archivos `.txt`.

#### 🔁 Retorno:
Devuelve una lista con los genes problemáticos, útil para análisis o correcciones posteriores.


In [ ]:
def verificar_docentes_asignados(cromosoma):
    errores = [gene for gene in cromosoma if gene[5] is None]

    if not errores:
        print("✅ Todos los docentes están correctamente asignados.")
    else:
        print(f"❌ Se encontraron {len(errores)} asignaciones sin docente.")
        for gene in errores:
            nivel, aula, dia, hora, codigo_materia, _ = gene
            print(f"- Sin docente → Nivel: {nivel}, Aula: {aula}, Día: {dia}, Hora: {hora}, Materia: {codigo_materia}")

    return errores  # retorna la lista por si necesitas trabajar con ella


In [ ]:
errores = verificar_docentes_asignados(cromosoma)


✅ Todos los docentes están correctamente asignados.


### 💾 9. Guardado del cromosoma en archivo `.txt`

La función `guardar_cromosoma_en_txt(cromosoma, nombre_archivo)` permite almacenar un cromosoma completo en un archivo plano para usos posteriores, como:

- Carga en notebooks de evaluación.
- Visualización en LaTeX.
- Backup de soluciones elite.

#### 📌 Detalles de funcionamiento:
- Recorre todos los genes del cromosoma.
- Escribe cada gen como una línea de texto en formato de tupla.
- El archivo resultante es fácilmente reutilizable con la función `cargar_cromosoma_desde_txt`.

#### ✅ Ejemplo de uso:
```python
guardar_cromosoma_en_txt(cromosoma, "Horario_LEV.txt")


In [ ]:
def guardar_cromosoma_en_txt(cromosoma, nombre_archivo):
    with open(nombre_archivo, "w", encoding="utf-8") as f:
        for gene in cromosoma:
            f.write(f"{gene}\n")
    print(f"📁 Cromosoma guardado correctamente en: {nombre_archivo}")


In [ ]:
guardar_cromosoma_en_txt(cromosoma, "Horario_LEV.txt")


📁 Cromosoma guardado correctamente en: Horario_LEV.txt
